In [3]:
from scipy.io import arff
import numpy as np
from sklearn import svm
from sklearn.metrics import confusion_matrix,accuracy_score

data,meta = arff.loadarff("PhishingData.arff")

#meta
data_ = data.astype(np.dtype([('SFH', 'int64'), ('popUpWidnow', 'int64'), ('SSLfinal_State', 'int64'), ('Request_URL', 'int64'), ('URL_of_Anchor', 'int64'), ('web_traffic', 'int64'), ('URL_Length', 'int64'), ('age_of_domain', 'int64'), ('having_IP_Address', 'int64'), ('Result', 'int64')]))

count_1 = 0; count_neg_1 = 0; count_0 = 0


partition_by_classes = {
    -1: [],
    0:[],
    1: []
}
neg_class = [point for point in data_ if point[-1] == -1]
zero_class = [point for point in data_ if point[-1] == 0]
one_class = [point for point in data_ if point[-1] == 1]




In [4]:
# Implementing K fold cross validation
def create_k_fold(data, k):
    k_subset = []
    start_point = 0
    for i in range(k):
        if i == k -1:
            k_subset.append(data[start_point:])
            continue
        rough_size = len(data) //k;
        subset = data[start_point:start_point + rough_size]
        k_subset.append(subset)
        start_point = start_point + rough_size
    
    return k_subset

k_neg_class = create_k_fold(neg_class, 10)
k_zero_class = create_k_fold(zero_class, 10)
k_one_class = create_k_fold(one_class, 10)


In [13]:
#implementing SVM classification
def SVM(k_one_class,k_neg_class,k_zero_class,n_fold,kernel_method,N_degree):
    ac = 0
    for i in range(n_fold):
        testing_index = i
        training_neg_class = k_neg_class[0:i+1] + k_neg_class[i+1:]
        training_zero_class =k_zero_class[0:i+1] + k_zero_class[i+1:]
        training_one_class = k_one_class[0:i+1] + k_one_class[i+1:]
        testing_neg_class = k_neg_class[i]
        testing_zero_class = k_zero_class[i]
        testing_one_class = k_one_class[i]
        train = training_neg_class + training_zero_class + training_one_class
        test = testing_neg_class +testing_zero_class+testing_one_class
        training_samples = []
        trainging_labels = []
        testing = []
        testing_labels = []


        for subset in train:
            for sample in subset:
                l = len(sample)
                training_samples.append(list(sample)[0:l-1])
                trainging_labels.append(sample[-1])

        for subset in test:
            l = len(subset)
            testing.append(list(subset)[0:l-1])
            testing_labels.append(subset[-1])
            
        clf = svm.SVC(decision_function_shape='ovo',kernel=kernel_method,coef0=1,degree=N_degree)
        clf.fit(training_samples, trainging_labels)
        pre = clf.predict(testing)
        mat = confusion_matrix(testing_labels,pre)
        accuracy = accuracy_score(testing_labels,pre)
        ac += accuracy
    print("average accuracy is: ",ac/n_fold)

In [16]:
SVM(k_one_class,k_neg_class,k_zero_class,10,'poly',2)

average accuracy is:  0.8802264189257791


In [17]:
SVM(k_one_class,k_neg_class,k_zero_class,10,'poly',3)

average accuracy is:  0.9119199918773478


In [18]:
SVM(k_one_class,k_neg_class,k_zero_class,10,'poly',4)

average accuracy is:  0.931190983856229


In [19]:
SVM(k_one_class,k_neg_class,k_zero_class,10,'poly',5)

average accuracy is:  0.9474109046603717


In [20]:
SVM(k_one_class,k_neg_class,k_zero_class,10,'poly',6)

average accuracy is:  0.9563001319930958


In [21]:
SVM(k_one_class,k_neg_class,k_zero_class,10,'poly',7)

average accuracy is:  0.960031475276678


In [22]:
SVM(k_one_class,k_neg_class,k_zero_class,10,'sigmoid',0)

average accuracy is:  0.7761904761904762


In [23]:
SVM(k_one_class,k_neg_class,k_zero_class,10,'linear',1)

average accuracy is:  0.8411361559549194
